In [1]:
from ipfn import ipfn
import pandas as pd
from PopSynthesis.Generator_data.generate_combine_census.utils import TRS

In [2]:
# decisions
using_BN = True
add_zero_cell = True

In [3]:
hh_marg = pd.read_csv("data/hh_marginals_ipu.csv", header=[0,1])
get_cols_by_first_level = lambda name_col: hh_marg.columns[hh_marg.columns.get_level_values(0) == name_col].to_list()
hh_marg = hh_marg.drop(columns=get_cols_by_first_level("sample_geog")).set_index(get_cols_by_first_level("zone_id")[0])
# remove the zero cell case
if not add_zero_cell:
    hh_marg[("hhinc", "Nil income")] = hh_marg[("hhinc", "Nil income")] +  hh_marg[("hhinc", "Negative income")]
    hh_marg = hh_marg.drop(columns=[("hhinc", "Negative income")])
hh_marg

hhsize                                   \
                                   1     2     3    4    5   6  7  8+   
(zone_id, Unnamed: 0_level_1)                                           
3000                           10393  8524  2026  656  118  27  4  58   
3002                            1035  1019   190  101   38   9  0  55   
3003                            1504  1591   476  223   40  11  7  55   
3004                            2566  2382   511  227   33   6  5  55   
3006                            5113  5033  1280  414   59  13  8  58   
...                              ...   ...   ...  ...  ...  .. ..  ..   
3990                               6    20     8    4    3   0  0  55   
3991                              37    65    16   19   10   9  0  55   
3992                             134   201    72   60   24   9  0  55   
3995                            1663  1834   629  451  228  73  8  58   
3996                             806  1176   251  299  137  31  6  59   

                                      dwelltype          ...      hhinc  \
                              Flat or Apartment Missing  ... Nil income   
(zone_id, Unnamed: 0_level_1)                            ...              
3000                                      21546     128  ...       1916   
3002                                       1877       9  ...         42   
3003                                       3187       6  ...        159   
3004                                       5741      27  ...        172   
3006                                      11783      40  ...        435   
...                                         ...     ...  ...        ...   
3990                                          0       0  ...          3   
3991                                          0       0  ...          6   
3992                                          0       0  ...          9   
3995                                         25       8  ...         87   
3996                                         12       4  ...         55   

                              totalvehs                              owndwell  \
                                      0     1     2    3   4+ Being Purchased   
(zone_id, Unnamed: 0_level_1)                                                   
3000                              14688  5445   861  397  415            2453   
3002                                478  1268   545   89   67             434   
3003                               1401  1937   420   93   56             797   
3004                               1198  3442   882  168   95            1207   
3006                               4725  5884  1026  193  150            2136   
...                                 ...   ...   ...  ...  ...             ...   
3990                                 13    12    30   25   16              16   
3991                                 20    46    66   42   37              51   
3992                                 12   153   218   93   79             229   
3995                                380  1949  1692  585  338            1326   
3996                                 88  1040  1207  298  132             690   

                                                                       
                              Being Rented Fully Owned Something Else  
(zone_id, Unnamed: 0_level_1)                                          
3000                                 16601        2284            468  
3002                                  1298         673             42  
3003                                  2548         463             99  
3004                                  3212        1268             98  
3006                                  8092        1537            213  
...                                    ...         ...            ...  
3990                                     0          22             58  
3991                                    17          67             76  
3992                         

In [4]:
hh_seed = pd.read_csv("data/hh_sample_ipu.csv")
# hh_seed = pd.read_csv("data/HH_pool.csv")
hh_seed = hh_seed.drop(columns=["sample_geog", "serialno"], errors="ignore")
hh_seed = hh_seed.value_counts().reset_index(name="total")
hh_seed["total"] = hh_seed["total"].astype(float)
hh_seed

,dwelltype,owndwell,hhinc,totalvehs,hhsize,total
0,Separate House,Fully Owned,500-649,1,1,370.0
1,Separate House,Fully Owned,1000-1249,2,2,366.0
2,Separate House,Being Purchased,2500-2999,2,4,272.0
3,Separate House,Fully Owned,1250-1499,2,2,236.0
4,Separate House,Fully Owned,650-799,1,2,234.0
...,...,...,...,...,...,...
3213,Separate House,Being Purchased,650-799,1,6,1.0
3214,Separate House,Being Purchased,650-799,0,6,1.0
3215,Separate House,Being Purchased,6000-7999,4+,7,1.0
3216,Separate House,Being Purchased,6000-7999,3,2,1.0


In [5]:
# this set the base
attrs = hh_seed.columns.to_list()
attrs.remove("total")
store_aggs = {}
for att in attrs:
    store_aggs[att] = hh_seed.groupby(att)["total"].sum()

# special case to handle zero-cell
if add_zero_cell:
    store_aggs["hhinc"].loc["Negative income"] = None # Place holder to handle later

In [6]:
resulted_syn = []
for zone_id in hh_marg.index:
    print(zone_id)
    marginals = hh_marg.loc[zone_id].to_dict()
    for att in attrs:
        for state in store_aggs[att].index:
            store_aggs[att].loc[state] = marginals[(att, state)]
    
    aggregates = [store_aggs[att] for att in attrs]
    dimensions = [[att] for att in attrs] 

    IPF = ipfn.ipfn(hh_seed.copy(), aggregates, dimensions)
    df = IPF.iteration()

    df["zone_id"] = zone_id

    # rounding using TRS
    known_tot = sum(marginals.values()) / len(attrs)
    # assert int(known_tot) == int(df["total"].sum()), f"known_tot: {known_tot}, df_total: {int(df['total'].sum())}"
    df["total"] = TRS(df["total"], known_tot)
    df = df[df["total"] > 0]

    resulted_syn.append(df)


3000
ipfn converged: convergence_rate not updating or below rate_tolerance
3002
ipfn converged: convergence_rate not updating or below rate_tolerance
3003
ipfn converged: convergence_rate not updating or below rate_tolerance
3004
ipfn converged: convergence_rate not updating or below rate_tolerance
3006
ipfn converged: convergence_rate not updating or below rate_tolerance
3008
ipfn converged: convergence_rate not updating or below rate_tolerance
3011
ipfn converged: convergence_rate not updating or below rate_tolerance
3012
ipfn converged: convergence_rate not updating or below rate_tolerance
3013
ipfn converged: convergence_rate not updating or below rate_tolerance
3015
ipfn converged: convergence_rate not updating or below rate_tolerance
3016
ipfn converged: convergence_rate not updating or below rate_tolerance
3018
ipfn converged: convergence_rate not updating or below rate_tolerance
3019
ipfn converged: convergence_rate not updating or below rate_tolerance
3020
ipfn converged: conv

In [7]:
final_result = pd.concat(resulted_syn)

In [8]:
assert set(final_result.set_index(attrs).index) <= set(hh_seed.set_index(attrs).index), f"synthesized comb must be subset of hh_seed"

In [9]:
if using_BN:
    final_result.to_csv("output/new_IPF_BN_pool.csv", index=False)
elif add_zero_cell:
    final_result.to_csv("output/new_IPF_wzero.csv", index=False)
else:
    final_result.to_csv("output/new_IPF_results_wo_zerocell.csv", index=False)

In [ ]:
# wo zerocell is 14m-37.7s
# with BN pool is 17m-32.8s
# with zerocell is 19m-32.8s